<a href="https://colab.research.google.com/github/RH00000/UH_RTS_Research_ML/blob/main/Non_skipping_cascades_simulated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upload the profiling NPZ file from local machine
from google.colab import files
uploaded = files.upload()

Saving abcd_profiling_validation_stats.npz to abcd_profiling_validation_stats.npz


In [3]:
# 1: Load profiling data
import numpy as np
import pandas as pd

data = np.load('abcd_profiling_validation_stats.npz')  # uses the uploaded file
confs = data['confs']    # shape (N,4)
times = data['times']    # shape (N,4)
oks   = data['oks']      # shape (N,4), bool
N, M = confs.shape       # N samples, M=4 models (A,B,C,D)

In [5]:
def simulate_static_idk_abcd(confs, times, oks,
                             thr_a, thr_b, thr_c):
    """
    Static IDK ABCD cascade:
      - Run A; if conf_a >= thr_a: exit at A
      - Else run B; if conf_b >= thr_b: exit at B
      - Else run C; if conf_c >= thr_c: exit at C
      - Else run D and exit there

    Returns:
      (accuracy, avg_time, frac_exit_at_A, frac_exit_at_B, frac_exit_at_C, frac_fallback_D)
    """
    total_time = 0.0
    correct = 0

    exit_a = 0
    exit_b = 0
    exit_c = 0
    exit_d = 0

    N = confs.shape[0]
    for j in range(N):
        # Classifier A
        conf_a = confs[j, 0]; t_a = times[j, 0]; ok_a = oks[j, 0]
        if conf_a >= thr_a:
            total_time += t_a
            correct += ok_a
            exit_a += 1
            continue

        # Classifier B
        conf_b = confs[j, 1]; t_b = times[j, 1]; ok_b = oks[j, 1]
        total_time += t_b
        if conf_b >= thr_b:
            correct += ok_b
            exit_b += 1
            continue

        # Classifier C
        conf_c = confs[j, 2]; t_c = times[j, 2]; ok_c = oks[j, 2]
        total_time += t_c
        if conf_c >= thr_c:
            correct += ok_c
            exit_c += 1
            continue

        # Fallback to D
        t_d = times[j, 3]; ok_d = oks[j, 3]
        total_time += t_d
        correct += ok_d
        exit_d += 1

    Nf = float(N)
    accuracy = correct / Nf
    avg_time = total_time / Nf
    return (
        accuracy,
        avg_time,
        exit_a / Nf,
        exit_b / Nf,
        exit_c / Nf,
        exit_d / Nf
    )


In [4]:
# Shared thresholds to evaluate
shared_thrs = [0.70, 0.75, 0.80, 0.85, 0.90, 0.95]

In [6]:
# Collect results
results = []
for st in shared_thrs:
    acc, avg_t, exit_a_frac, exit_b_frac, exit_c_frac, exit_d_frac = simulate_static_idk_abcd(
        confs, times, oks, st, st, st
    )
    results.append({
        "shared_thr": st,
        "accuracy": acc,
        "avg_time_ms": avg_t * 1000,
        "exit_A_frac": exit_a_frac,
        "exit_B_frac": exit_b_frac,
        "exit_C_frac": exit_c_frac,
        "exit_D_frac": exit_d_frac,
    })

In [7]:
# Convert to DataFrame and output
import pandas as pd
df_static = pd.DataFrame(results)
print(df_static)

   shared_thr  accuracy  avg_time_ms  exit_A_frac  exit_B_frac  exit_C_frac  \
0        0.70    0.7888    11.471352       0.6044       0.1360       0.0704   
1        0.75    0.7932    12.885091       0.5608       0.1344       0.0688   
2        0.80    0.7988    14.223134       0.5176       0.1308       0.0756   
3        0.85    0.8020    15.817357       0.4712       0.1264       0.0752   
4        0.90    0.8032    17.690344       0.4196       0.1236       0.0684   
5        0.95    0.8008    20.275887       0.3464       0.1112       0.0704   

   exit_D_frac  
0       0.1892  
1       0.2360  
2       0.2760  
3       0.3272  
4       0.3884  
5       0.4720  
